<h1>Figure 1C</h1>

In [1]:
#fname <- "https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt"

In [42]:
rm(list = ls())

In [43]:
dfile <- 'data/SraRunTable.noCram.noExome.noWGS.totalRNA.txt'

In [55]:
meta.data <- read.csv(dfile, header=FALSE)

In [56]:
#rnaseq <- dat[which(dat$SMAFRZE=="RNASEQ"),]


In [57]:
dim(meta.data)

[1] 9785   79

In [58]:
head(meta.data)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79
,<fct>,<fct>,<fct>,<int>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<lgl>,<fct>,<fct>,<lgl>
1,SRR2157314,RNA:Total RNA,RNA-Seq,500,49773955000,PRJNA75899,SAMN03878353,GTEx,GTEX-T6MN-0226-SM-5S2SO,Adipose - Subcutaneous,⋯,RNA,P-RNA-0009,P-RNA-0009,NA,NA,,NA,,,NA
2,SRR2157460,RNA:Total RNA,RNA-Seq,500,48762020000,PRJNA75899,SAMN03877433,GTEx,GTEX-XLM4-0011-R10A-SM-5S2TN,Brain - Frontal Cortex (BA9),⋯,RNA,P-RNA-0009,P-RNA-0009,NA,NA,,NA,,,NA
3,SRR2164562,RNA:Total RNA,RNA-Seq,500,50374455500,PRJNA75899,SAMN03877762,GTEx,GTEX-S32W-0726-SM-5S2T3,Thyroid,⋯,RNA,P-RNA-0009,P-RNA-0009,NA,NA,,NA,,,NA
4,SRR2164608,RNA:Total RNA,RNA-Seq,500,49373870000,PRJNA75899,SAMN03877938,GTEx,GTEX-VUSG-2426-SM-5S2S2,Adipose - Subcutaneous,⋯,RNA,P-RNA-0009,P-RNA-0009,NA,NA,,NA,,,NA
5,SRR2170135,RNA:Total RNA,RNA-Seq,500,46078920500,PRJNA75899,SAMN03878198,GTEx,GTEX-T6MN-0826-SM-5S2SK,Lung,⋯,RNA,P-RNA-0009,P-RNA-0009,NA,NA,,NA,,,NA
6,SRR2170173,RNA:Total RNA,RNA-Seq,500,45386225000,PRJNA75899,SAMN03878158,GTEx,GTEX-XXEK-2426-SM-5S2S8,Adipose - Subcutaneous,⋯,RNA,P-RNA-0009,P-RNA-0009,NA,NA,,NA,,,NA


In [59]:
tissue.sets<-unique(as.character(meta.data$body_site_s))


In [60]:
gtex_url <- 'https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct.gz'

In [63]:
gtex_gct_dest <- 'data/gtexgct.gz'
download.file(gtex_url,destfile=gtex_gct_dest)

In [65]:
#untar(gtex_gct_dest)
gtex_gct='data/gtexgct'

In [67]:
gtex_pheno_url <- 'https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt'
gtex_pheno <-'data/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt'
download.file(gtex_pheno_url, destfile=gtex_pheno)

In [66]:
for (tissue.set in tissue.sets)
{

expression.mat<-read.table(gtex_gct, nrow=1,sep='\t',header=T,skip=2)

colnames.expression.mat<-colnames(expression.mat)

meta.data <- read.csv(dfile, header=FALSE)
    
if (length(table(meta.data$sex_s[meta.data$body_site_s %in% tissue.set]))==1)

  next

meta.data$Sample_Name_s<-gsub('-','\\.',meta.data$Sample_Name_s)

meta.data<-meta.data[meta.data$Sample_Name_s %in% colnames(expression.mat),]

col.in.tissue<-c()

if (sum(meta.data$body_site_s %in% tissue.set)==0)

  next                                                                                                                                                                                            

for (col in colnames.expression.mat)

  col.in.tissue<-c(col.in.tissue, (col %in% meta.data$Sample_Name_s) && (meta.data$body_site_s[which(meta.data$Sample_Name_s==col)] %in% tissue.set))

expression.mat<-read.table(gtex_gct, colClasses = ifelse(col.in.tissue,"numeric","NULL"),sep='\t',header=T,skip=2)

write.table(expression.mat,paste('data/expression',tissue.set,'.txt',sep=''),sep='\t',quote = F)

}

In [68]:
for (tissue.set in tissue.sets)
{
  all.genes<-read.table(gtex_gct,sep='\t',header=T,skip=2,colClasses = c(rep("character", 2), rep("NULL", 11688)))
if (file.size(paste('gene_expression/expression',tissue.set,'.txt',sep=''))<=1)

  next

x <- read.delim(paste('gene_expression/expression',tissue.set,'.txt',sep=''))

#x<-x[all.genes$Description %in% our.genes,]

#all.genes<-all.genes[all.genes$Description %in% our.genes,2]

x<-x[!duplicated(all.genes[,'Description']),]

all.genes<-all.genes[!duplicated(all.genes[,'Description']),'Description']

rownames(x)<-all.genes

#meta.data<-read.table('/Users/karleg/Dimorph/2017December8GTExRNASeqSRARunTable.txt',sep='\t',header=TRUE)

#meta.data$Sample_Name_s<-gsub('-','\\.',meta.data$Sample_Name_s)

#meta.data<-meta.data[meta.data$Sample_Name_s %in% colnames(x),]

    
pheno<-read.csv(gtex_pheno,sep='\t')

pheno$SUBJID<-gsub('-','\\.',pheno$SUBJID)

sex<-c()  #the value 2 in the phenotypic data is the values that is 1 in design.  This value correslonds to female.

age<-c()

cod<-c()

for (col in colnames(x))
{
  sex<-c(sex,pheno$SEX[which(pheno$SUBJID==paste(unlist(strsplit(col,'\\.'))[c(1,2)],collapse='.'))])

  age<-c(age,pheno$AGE[which(pheno$SUBJID==paste(unlist(strsplit(col,'\\.'))[c(1,2)],collapse='.'))])

  cod<-c(cod,pheno$DTHHRDY[which(pheno$SUBJID==paste(unlist(strsplit(col,'\\.'))[c(1,2)],collapse='.'))])

}

if (length(table(sex))==1)

  next

cod[is.na(cod)]=0
#group <- meta.data$sex_s

y <- DGEList(counts=x,group=factor(sex))

y <- calcNormFactors(y)

groups<-sex

keep.events<-rep(T,nrow(y))

for (group in c(1,2))

  keep.events<-keep.events & (rowSums(cpm(y[,groups %in% group]) > 1) >= 0.25*min(table(groups)))

if (sum(keep.events)==0)

  next

y<-y[keep.events,]  #if only using the logFC to compare with AS then do not screen

design <- model.matrix(~factor(sex))

v <- voom(y, design)

fit <- lmFit(v, design)

fit <- eBayes(fit, robust=TRUE)

res=topTable(fit, coef='factor(sex)2',number=nrow(y))

write.table(res,paste('data/DE_result_',tissue.set,'.txt',sep=''),sep='\t',quote = F)

}
#edgeR

#y <- estimateDisp(y,design)

#fit <- glmFit(y,design)

#lrt <- glmLRT(fit,coef='factor(sex)2')

#hist(lrt$table$PValue)    